In [1]:
import pandas as pd
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [2]:
import MySQLdb
import MySQLdb.cursors
 
database = MySQLdb.connect(host = "work-point.cckrazp59xhr.ap-south-1.rds.amazonaws.com", user = "work_admin", passwd = "YpFyCwcdpKAswevhXMjy", db = "work_point_db", cursorclass=MySQLdb.cursors.DictCursor)
cursor = database.cursor()

In [3]:
cursor.execute("SELECT * FROM client_job")
job_list = cursor.fetchall()
job_list = list(job_list)

In [4]:
cursor.execute("SELECT * FROM client_skill")
skill_list = cursor.fetchall()
skill_list = list(skill_list)

In [5]:
cursor.execute("SELECT * FROM client_skill_job")
job_skill_list = cursor.fetchall()
job_skill_list = list(job_skill_list)

In [6]:
job_df = pd.DataFrame.from_dict(job_list)
skill_df =  pd.DataFrame.from_dict(skill_list)
skill_df = skill_df.rename(columns={'id':'skill_id'})
skill_df = skill_df.rename(columns={'name':'skill_name'})
job_skill_df = pd.DataFrame.from_dict(job_skill_list)
job_skill_df = job_skill_df.drop(['id'],axis='columns')
job_df = job_df.rename(columns={'id':'job_id'})

In [7]:
job_skill_combined = pd.merge(job_df, job_skill_df, on='job_id')

In [8]:
combine_df = pd.merge(job_skill_combined,skill_df, on='skill_id')

In [9]:
combine_df.drop(['is_completed','client_id','price','posted_date','skill_id'],axis='columns',inplace=True)

In [10]:
username = 'arjun007'
skills = ['video']

In [11]:
new_row = pd.DataFrame.from_dict({'title':username, 'skill_name':skills})
combine_df = pd.concat([combine_df, new_row])

In [12]:
combined_skill_list=[]
for i in combine_df['title']:
    temp = list(combine_df[combine_df['title']==i]['skill_name'])
    combined_skill_list.append(temp)

In [13]:
combine_df['skill_list'] = combined_skill_list

In [14]:
combine_df.drop(['skill_name'],axis='columns',inplace=True)

In [15]:
combine_df.drop_duplicates(subset=['title'],keep="first",inplace=True)

In [16]:
def create_soup(x):
    return ' '.join(x['skill_list'])

In [17]:
combine_df['soup'] = combine_df.apply(create_soup, axis=1)

In [18]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(combine_df['soup'])

In [19]:
combine_df = combine_df.reset_index()

In [20]:
indices = pd.Series(combine_df.index, index=combine_df['title'])

In [21]:
sig = sigmoid_kernel(count_matrix, count_matrix)

In [22]:
def give_rec(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[:10]
    job_indices = [i[0] for i in sig_scores]
    return combine_df['job_id'].iloc[job_indices].drop(combine_df[combine_df['title'] == username].index[0]).astype(int)

In [23]:
list(give_rec('arjun007'))

[5, 4]